In [1]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

In [2]:
!pip3 install Pysyft

In [3]:
!pip install syft=="0.2.2a1"
!pip install syft_proto=="0.1.1a1.post17"

     |████████████████████████████████| 337kB 8.7MB/s 
     |████████████████████████████████| 112kB 36.9MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 773.1MB 22kB/s 
     |████████████████████████████████| 2.1MB 30.0MB/s 
     |████████████████████████████████| 1.8MB 26.8MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 10.2MB 16.0MB/s 
     |████████████████████████████████| 532kB 44.6MB/s 
     |████████████████████████████████| 61kB 514kB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
  Created wheel for phe: filename=phe-1.4.0-py2.py3-none-any.whl size=37362 sha256=b6459795114b7389d197403d6296371c26694300c2795431bf63daa9e95c4342
  Stored in directory: /root/.cache/pip/wheels/f8/dc/36/dcb6bf0f1b9907e7b710ace63e64d08e7022340909315fdea4
  Created wheel for zstd: filename=zstd-1.5.0.2-cp37-cp37m-linux_x86_64.whl size=1672598 sha256=09c4fa1f1f246682597a9fd6a25075c0a58bf5e76

     |████████████████████████████████| 40kB 4.4MB/s 
  Found existing installation: syft-proto 0.5.3
    Uninstalling syft-proto-0.5.3:
      Successfully uninstalled syft-proto-0.5.3


In [2]:
import syft as sy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
anne = sy.VirtualWorker(hook, id="anne")

In [4]:
x = torch.Tensor([2,2,2]).send(bob)
print(x)

(Wrapper)>[PointerTensor | me:86956231882 -> bob:3094165622]


In [ ]:
def load_data():
    
    '''<--Load CIFAR dataset from torch vision module distribute to workers using PySyft's Federated Data loader'''
    

    federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.CIFAR10('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

    test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)
    
    return federated_train_loader,test_loader

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = True
        self.seed = 1
        self.log_interval = 200
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
#<--Load federated training data and test data
federated_train_loader,test_loader=load_data()

#<--Create Neural Network model instance
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) #<--TODO momentum is not supported at the moment

#<--Train Neural network and validate with test set after completion of training every epoch
for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "cifar10_cnn.pt")


0it [00:00, ?it/s]



Failed download. Trying https -> http instead. Downloading http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ../data/cifar-10-python.tar.gz


0it [00:00, ?it/s]

  0%|          | 49152/170498071 [00:00<06:26, 440993.81it/s]

  0%|          | 237568/170498071 [00:00<04:57, 571926.19it/s]

  0%|          | 425984/170498071 [00:00<03:55, 722926.63it/s]

  0%|          | 614400/170498071 [00:00<03:12, 884044.47it/s]

  0%|          | 802816/170498071 [00:00<02:41, 1047902.51it/s]

  1%|          | 991232/170498071 [00:00<02:20, 1207996.21it/s]

  1%|          | 1179648/170498071 [00:00<02:05, 1346896.32it/s]

  1%|          | 1368064/170498071 [00:00<01:55, 1470601.01it/s]

  1%|          | 1556480/170498071 [00:00<01:48, 1560228.34it/s]

  1%|          | 1744896/170498071 [00:01<01:43, 1635685.54it/s]

  1%|          | 1933312/170498071 [00:01<01:39, 1700856.71it/s]

  1%|          | 2121728/170498071 [00:01<01:36, 1744656.16it/s]

  1%|▏         | 2310144/170498071 [00:01<01:37, 1732735.40it/s]

  1%|▏         | 2490368/170498071 [00:01<01:37, 1714860.01it/s]

  2%|▏         | 2670592/170498071 [00:01<01:37, 1723673.82it/s]

 

Extracting ../data/cifar-10-python.tar.gz to ../data




170500096it [01:39, 9230111.01it/s]                               

Train Epoch: 1 [0/50048 (0%)]	Loss: 2.310682
Train Epoch: 1 [12800/50048 (26%)]	Loss: 2.301998
Train Epoch: 1 [25600/50048 (51%)]	Loss: 2.293859
Train Epoch: 1 [38400/50048 (77%)]	Loss: 2.298970

Test set: Average loss: 2.2804, Accuracy: 1528/10000 (15%)

Train Epoch: 2 [0/50048 (0%)]	Loss: 2.285189
Train Epoch: 2 [12800/50048 (26%)]	Loss: 2.214281
Train Epoch: 2 [25600/50048 (51%)]	Loss: 2.174301
Train Epoch: 2 [38400/50048 (77%)]	Loss: 1.991645

Test set: Average loss: 1.9646, Accuracy: 2947/10000 (29%)

Train Epoch: 3 [0/50048 (0%)]	Loss: 1.924952
Train Epoch: 3 [12800/50048 (26%)]	Loss: 1.949415
Train Epoch: 3 [25600/50048 (51%)]	Loss: 1.895669
Train Epoch: 3 [38400/50048 (77%)]	Loss: 1.820386

Test set: Average loss: 1.7785, Accuracy: 3498/10000 (35%)

Train Epoch: 4 [0/50048 (0%)]	Loss: 1.611385
Train Epoch: 4 [12800/50048 (26%)]	Loss: 1.611142
Train Epoch: 4 [25600/50048 (51%)]	Loss: 1.711248
Train Epoch: 4 [38400/50048 (77%)]	Loss: 1.616658

Test set: Average loss: 1.6442, Accu